# Implement a basic RAG pipeline with Haystack 2.0

This notebook demonstrates how to implement a basic Retrieval Augmented Generation (RAG) pipeline using the `Haystack` orchestration framework and HuggingFace e.g `SmolLM` LLM Models. You can select your difficulty by trying out different models, knowledge base, tasks, or reimplementing in a different frameworks or even without one.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: </h2>
    <p>Implement a RAG pipeline and tailor it to your needs. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Follow step-by-step instructions. Try different embedding and generative model variants.</p>
    <p>🐕 Try different documents and evaluation questions.</p>
    <p>🦁 Reimplement part of or the whole pipeline with different modules or frameworks.</p>
</div>


### Dependencies  
- **`haystack-ai`**: Core framework for building RAG pipelines.  
- **`wikipedia`**: Downloads data from Wikipedia (optional; any document source can be used, such as PDFs, web pages, or local text files).  
- **`sentence_transformers`**: Generates embeddings for document chunks.  
- **`transformers`**: Utilizes open-source LLMs for generation.  
- **`accelerate`** and **`bitsandbytes`**: Enable efficient, low-memory model inference using quantized versions.  


In [ ]:
! pip install wikipedia haystack-ai transformers accelerate bitsandbytes sentence_transformers

In [ ]:
from IPython.display import Image
import torch
import random

from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

from haystack.components.generators import HuggingFaceLocalGenerator

from huggingface_hub import login
login()

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

model_llm = "HuggingFaceTB/SmolLM2-135M-Instruct"
model_emb = "sentence-transformers/all-MiniLM-L6-v2"

/home/duydl/Miniconda3/envs/py310/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


### Indexing Pipeline

We are going to download the Wikipedia pages related to , using the python library `wikipedia`.

These pages are converted into Haystack Documents